In [62]:
import pandas as pd
metadata_df = pd.read_csv("./HAM10000_metadata")

In [63]:
import os
import shutil

# Define the main directory where you want to store the organized images
main_directory = './dataset/train'

# Define the directories where your images are currently located
image_directories = ['./HAM10000_images_part_1', './HAM10000_images_part_2']

# Identify unique classes in the 'dx' column
unique_classes = metadata_df['dx'].unique()

# Create a directory for each unique class
for class_label in unique_classes:
    class_directory = os.path.join(main_directory, class_label)
    os.makedirs(class_directory, exist_ok=True)

# Iterate over the metadata DataFrame and move each image to the corresponding class directory
for index, row in metadata_df.iterrows():
    image_id = row['image_id']
    class_label = row['dx']
    
    # Define destination path
    destination_path = os.path.join(main_directory, class_label, f"{image_id}.jpg")
    
    # Check both source folders for the image
    for image_directory in image_directories:
        source_path = os.path.join(image_directory, f"{image_id}.jpg")
        
        # If the source path exists, move the image and break out of the loop
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)
            break


In [64]:
# splitting validation set
import os
import shutil
from sklearn.model_selection import train_test_split
train_dir = './dataset/train'
val_dir = './dataset/validation'

if not os.path.exists(val_dir):
    os.makedirs(val_dir)
for class_name in os.listdir(train_dir):
    # Define class-specific directories for train and validation sets
    class_train_dir = os.path.join(train_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)
    
    # Get the list of image filenames for the class
    images = os.listdir(class_train_dir)
    
    # Split the list of images into training and validation subsets
    train_images, val_images = train_test_split(images, test_size=0.3, random_state=42)  # 30% of data to validation set
    
    # If the class-specific validation directy does not exist, create it
    if not os.path.exists(class_val_dir):
        os.makedirs(class_val_dir)
    
    # Move the validation images to the validation directory
    for img in val_images:
        src = os.path.join(class_train_dir, img)
        dst = os.path.join(class_val_dir, img)
        shutil.move(src, dst)

        

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the training generator with data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rotation_range=20,        # rotate the image 20 degrees
    width_shift_range=0.1,    # Shift the pic width by a max of 10%
    height_shift_range=0.1,   # Shift the pic height by a max of 10%
    shear_range=0.1,          # Shear means cutting away part of the image (max 10%)
    zoom_range=0.2,           # Zoom in by 20% max
    horizontal_flip=True,     # Allow horizontal flipping
    fill_mode='nearest'       # Fill in missing pixels with the nearest filled value
)


# Define the validation generator with only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    "./dataset/train", # path to the training data directory
    target_size=(128 , 128), # dimensions to which all images will be resized
    batch_size=15, 
    class_mode='categorical' # or 'categorical' based on your problem
)

val_generator = val_datagen.flow_from_directory(
    "./dataset/validation", # path to the validation data directory
    target_size=(128,128), 
    batch_size=15, 
    class_mode='categorical' # or 'categorical' based on your problem
)


Found 7007 images belonging to 7 classes.
Found 3008 images belonging to 7 classes.


In [66]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.regularizers import l2

model = Sequential()


model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 126, 126, 16)      448       
                                                                 
 batch_normalization_42 (Ba  (None, 126, 126, 16)      64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 63, 63, 16)        0         
 ng2D)                                                           
                                                                 
 dropout_34 (Dropout)        (None, 63, 63, 16)        0         
                                                                 
 conv2d_35 (Conv2D)          (None, 61, 61, 32)        4640      
                                                                 
 batch_normalization_43 (Ba  (None, 61, 61, 32)       

In [67]:
from tensorflow.keras.optimizers.legacy import Adam
from keras import callbacks

epochs = 20  # A small number of epochs
lr = 0.001   # A larger learning rate
total_train = len(train_generator.filenames)
total_val = len(val_generator.filenames)

model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=total_train // 15,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=total_val // 15,
    callbacks=[early_stopping]
)
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

/var/folders/jw/1n94bt095gvd97f9yxx9w8n80000gn/T/ipykernel_2551/2831364900.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
467/467 [==============================] - 110s 234ms/step - loss: 4.2330 - accuracy: 0.4645 - val_loss: 5.1016 - val_accuracy: 0.1097
Epoch 2/20
467/467 [==============================] - 107s 229ms/step - loss: 3.4706 - accuracy: 0.6051 - val_loss: 7.5295 - val_accuracy: 0.1097
Epoch 3/20
 97/467 [=====>........................] - ETA: 1:09 - loss: 3.1005 - accuracy: 0.6498

KeyboardInterrupt: 